In [ ]:
import sys
!{sys.executable} -m pip install "dask[complete]"

In [10]:
# import the necessary packages
import argparse
import cv2
import sys
import pandas as pd
import dask.dataframe as dd
# Function to load and preprocess the image
# Function to compute motion blur score using DWT
import pywt
import numpy as np



# construct the argument parse and parse the arguments
def get_is_blur(imagePath):
    threshold = 100
    # load the image, convert it to grayscale, and compute the
    # focus measure of the image using the Variance of Laplacian
    # method
    image = cv2.imread(imagePath)
    # Check if the image is loaded properly
    if image is None:
        print(f"Error: Could not load image {imagePath}")
        return 0

    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()





def compute_motion_blur_score_from_wavelet_transformation(image_path):
    threshold = 35
    # Load image in grayscale
    image = cv2.imread(image_path)
    if image is None:
        return 0
    # Convert image to grayscale
    Y = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    
    M, N = Y.shape
    
    # Crop input image to be 3 divisible by 2
    Y = Y[0:int(M/16)*16, 0:int(N/16)*16]
    
    # Step 1, compute Haar wavelet of input image
    LL1,(LH1,HL1,HH1)= pywt.dwt2(Y, 'haar')
    # Another application of 2D haar to LL1
    LL2,(LH2,HL2,HH2)= pywt.dwt2(LL1, 'haar') 
    # Another application of 2D haar to LL2
    LL3,(LH3,HL3,HH3)= pywt.dwt2(LL2, 'haar')
    
    # Construct the edge map in each scale Step 2
    E1 = np.sqrt(np.power(LH1, 2)+np.power(HL1, 2)+np.power(HH1, 2))
    E2 = np.sqrt(np.power(LH2, 2)+np.power(HL2, 2)+np.power(HH2, 2))
    E3 = np.sqrt(np.power(LH3, 2)+np.power(HL3, 2)+np.power(HH3, 2))
    
    M1, N1 = E1.shape

    # Sliding window size level 1
    sizeM1 = 8
    sizeN1 = 8
    
    # Sliding windows size level 2
    sizeM2 = int(sizeM1/2)
    sizeN2 = int(sizeN1/2)
    
    # Sliding windows size level 3
    sizeM3 = int(sizeM2/2)
    sizeN3 = int(sizeN2/2)
    
    # Number of edge maps, related to sliding windows size
    N_iter = int((M1/sizeM1)*(N1/sizeN1))
    
    Emax1 = np.zeros((N_iter))
    Emax2 = np.zeros((N_iter))
    Emax3 = np.zeros((N_iter))
    
    
    count = 0
    
    # Sliding windows index of level 1
    x1 = 0
    y1 = 0
    # Sliding windows index of level 2
    x2 = 0
    y2 = 0
    # Sliding windows index of level 3
    x3 = 0
    y3 = 0
    
    # Sliding windows limit on horizontal dimension
    Y_limit = N1-sizeN1
    
    while count < N_iter:
        # Get the maximum value of slicing windows over edge maps 
        # in each level
        Emax1[count] = np.max(E1[x1:x1+sizeM1,y1:y1+sizeN1])
        Emax2[count] = np.max(E2[x2:x2+sizeM2,y2:y2+sizeN2])
        Emax3[count] = np.max(E3[x3:x3+sizeM3,y3:y3+sizeN3])
        
        # if sliding windows ends horizontal direction
        # move along vertical direction and resets horizontal
        # direction
        if y1 == Y_limit:
            x1 = x1 + sizeM1
            y1 = 0
            
            x2 = x2 + sizeM2
            y2 = 0
            
            x3 = x3 + sizeM3
            y3 = 0
            
            count += 1
        
        # windows moves along horizontal dimension
        else:
                
            y1 = y1 + sizeN1
            y2 = y2 + sizeN2
            y3 = y3 + sizeN3
            count += 1
    
    # Step 3
    EdgePoint1 = Emax1 > threshold;
    EdgePoint2 = Emax2 > threshold;
    EdgePoint3 = Emax3 > threshold;
    
    # Rule 1 Edge Pojnts
    EdgePoint = EdgePoint1 + EdgePoint2 + EdgePoint3
    
    n_edges = EdgePoint.shape[0]
    
    # Rule 2 Dirak-Structure or Astep-Structure
    DAstructure = (Emax1[EdgePoint] > Emax2[EdgePoint]) * (Emax2[EdgePoint] > Emax3[EdgePoint]);
    
    # Rule 3 Roof-Structure or Gstep-Structure
    
    RGstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax1[i] < Emax2[i] and Emax2[i] < Emax3[i]:
            
                RGstructure[i] = 1
                
    # Rule 4 Roof-Structure
    
    RSstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax2[i] > Emax1[i] and Emax2[i] > Emax3[i]:
            
                RSstructure[i] = 1

    # Rule 5 Edge more likely to be in a blurred image 

    BlurC = np.zeros((n_edges));

    for i in range(n_edges):
    
        if RGstructure[i] == 1 or RSstructure[i] == 1:
        
            if Emax1[i] < threshold:
            
                BlurC[i] = 1                        
        
    # Step 6
    Per = np.sum(DAstructure)/np.sum(EdgePoint)
    
    # Step 7
    if (np.sum(RGstructure) + np.sum(RSstructure)) == 0:
        
        BlurExtent = 100
    else:
        BlurExtent = np.sum(BlurC) / (np.sum(RGstructure) + np.sum(RSstructure))
    
    if Per < 0.007:
        classification = True
    else:
        classification = False
    print(Per)
    return classification
    

In [11]:
import glob

# Define the pattern to match all .jpg files in the nested directories
pattern = "/home/research/kilasar_sentinal_wepon_detection/data/*/*/*/*.jpg"

# Get all matching file paths
all_files = glob.glob(pattern)

# Define the folder to exclude
exclude_folder = "/n/"

# Filter out files that contain the exclude_folder in their path
filtered_files = [f for f in all_files if exclude_folder not in f]

# Print the filtered list of files
print(len(filtered_files))

34008


In [12]:

df = pd.DataFrame(filtered_files, columns=["image_paths"])
df.shape

(34008, 1)

In [14]:
compute_motion_blur_score_from_wavelet_transformation("/home/research/kilasar_sentinal_wepon_detection/data/2/train/images/video_fb784f02-8da0-4e89-932d-61f9f7fbcbf1_frame_0364.jpg")

0.0009439899307740718


True

In [ ]:


# Assuming df is your existing Pandas DataFrame
ddf = dd.from_pandas(df, npartitions=40)  # Adjust npartitions based on your CPU cores

# ddf["blurriness"] = ddf["image_paths"].map_partitions(lambda df: df.apply(get_is_blur))


In [ ]:
ddf["blurriness_from_wavelet"] = ddf["image_paths"].map_partitions(lambda df: df.apply(compute_motion_blur_score_from_wavelet_transformation))
# Compute the results and convert back to Pandas DataFrame
result_df = ddf.compute()

In [ ]:
import matplotlib.pyplot as plt

# Plot the histogram
result_df["blurriness"].plot(kind='hist', bins=30, edgecolor='black')

# Add labels and title
plt.xlabel('Blurriness')
plt.ylabel('Frequency')
plt.title('Histogram of Blurriness Scores')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the histogram
result_df["blurriness_from_wavelet"].plot(kind='hist', bins=30, edgecolor='black')

# Add labels and title
plt.xlabel('Blurriness')
plt.ylabel('Frequency')
plt.title('Histogram of Blurriness Scores')

# Show the plot
plt.show()

In [ ]:
blur_df = result_df[(result_df["blurriness_from_wavelet"]==True)]

In [ ]:
list_of_blurry_images = blur_df["image_paths"].to_list()
batch_size = 9
num_batches = (len(list_of_blurry_images) + batch_size - 1) // batch_size  # Calculate number of batches

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(list_of_blurry_images))
    
    # Plotting the current batch of images in a grid
    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    
    for i, ax in enumerate(axes.flatten()):
        if start_idx + i < end_idx:
            image_path = list_of_blurry_images[start_idx + i]
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for displaying
            ax.imshow(image)
            ax.set_title(f"Blurriness: {result_df[result_df['image_paths'] == image_path]['blurriness'].values[0]:.2f}")
            ax.axis('off')
        else:
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()